In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# 1. 데이터 준비
df = pd.read_csv('C:/Users/user/Downloads/temp_ngram_div20.csv')
# 10000개 샘플 데이터
df = df.sample(n=10000)
df = df.reset_index(drop = True)
word_list = df.values.tolist()

x_data = [i[0] for i in word_list]
y_data = [i[1] for i in word_list]

# 2. 특성 추출 (형태소 분석 및 n-그램)
# mecab = Mecab()
# tagger = mecab.pos
# mecab_tokenize = lambda text: [token for token, pos in tagger(text) if pos == "NNG" or pos == "VA" or pos == "VAX" or pos == "MAG" or pos == "VV"]
vectorizer = CountVectorizer(min_df=15)
vectorizer.fit(x_data)
x_data = vectorizer.transform(x_data)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=10, stratify=y_data)

# 3. 나이브 베이즈 모델 학습
clf = MultinomialNB()
clf.fit(x_train, y_train)

# 4. 클래스별 확률 데이터 프레임 생성
proba_array = clf.predict_proba(x_data)
proba_df = pd.DataFrame(proba_array, columns=[f'prob_{cls}' for cls in clf.classes_])
df_with_proba = pd.concat([df, proba_df], axis = 1)

# 5. 클래스별 확률 및 모델 정확도 출력
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

# 6. 모델 과적합 출력
print(clf.score(x_train, y_train))
print(clf.score(x_test, y_test))

              precision    recall  f1-score   support

           0       0.42      0.66      0.51       413
           1       0.28      0.11      0.16       249
           2       0.35      0.26      0.30       338

    accuracy                           0.39      1000
   macro avg       0.35      0.34      0.32      1000
weighted avg       0.36      0.39      0.35      1000

0.45944444444444443
0.388


In [ ]:
import pandas as pd

In [ ]:
def process_dataframe(df):
    # polarity_score 계산
    df_with_proba['score'] = df_with_proba['prob_2'] / df_with_proba['prob_0']

    # 중복행 제거
    df = df_with_proba.drop_duplicates()

    # 소수점 2자리수까지만 남기기
    df['score'] = df['score'].round(2)

    # 중립값 제거하기
    df = df[(df['score'] > 1.3) | (df['score'] < 0.77)]

    # 매파/비둘기파 라벨링
    df['polarity'] = df['score'].apply(lambda x: 'hawkish' if x > 1.3 else 'dovish')

    # 텍스트랑 라벨링한 열만 남기기
    df = df[['text', 'polarity']]

    # 사전화하기
    dic = df.set_index('text')['polarity'].to_dict()

    # 세미콜론 제거하기
    new_key = [i.replace(';', ', ') for i in list(dic.keys())]
    old_key = list(dic.keys())

    # 라벨에 따라 사전 만들기
    new_dict = {new: dic[old] for new, old in zip(new_key, old_key)}
    return new_dict
